In [2]:
import os
from langchain import vectorstores, document_loaders, embeddings, chat_models, chains, prompts
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
import json
from langchain.prompts import PromptTemplate, ChatPromptTemplate


load_dotenv('../.env',override=True)


True

In [1]:
model_name = 'text-embedding-ada-002'
embeddings = OpenAIEmbeddings(model=model_name)
vectorstore_tad = FAISS.load_local('../faiss_db/tad-db', embeddings, allow_dangerous_deserialization=True)
vectorstore_tad_title = FAISS.load_local('../faiss_db/tad-db-title-key', embeddings, allow_dangerous_deserialization=True)
llm_gpt3 = ChatOpenAI(
    model_name='gpt-3.5-turbo-1106',
    temperature=0.2
)
llm_gpt4 = ChatOpenAI(
    model_name='gpt-4',
    temperature=0.0
)

NameError: name 'OpenAIEmbeddings' is not defined

In [4]:
def generate_context_from_theme(course_theme, countext_count=1):
    documents = vectorstore_tad.max_marginal_relevance_search(
        query=course_theme, k=countext_count, fetch_k=50, 
    )
    result = []
    for i in documents:
        course_data = (
            'BEGINING_OF_COURSE_DATA\n'
            f'A kurzus címe: {documents[0].metadata["course_title"]},\n'
            f'Kreditértéke: {documents[0].metadata["course_credit"]},\n'
            f'Követelmények: {documents[0].metadata["course_type"]},\n'
            f'Forrás: {documents[0].metadata["source"]},\n'
            f'A tárgy leírása: {documents[0].metadata["course_text"]}\n'
            'END_OF_COURSE_DATA'
        )
        result.append(course_data)
    return "".join(result)

def process_generation(result):
    return result.split('A válasz:')

def generate_question_answer_context(course_theme,question_count=1, context_count=1, llm = llm_gpt3):

    context = generate_context_from_theme(course_theme, context_count)
    qa = PromptTemplate.from_template(
            template=(
                'BME hallgatóinak készülő QA rendszer tesztelője vagyok. '
                'Jelenleg a tantárgyakkal kapcsolatos információkat tesztelem. '
                'A tárgyakkal kapcsolatos információk az END_OF_COURSE_DATA és BEGINING_OF_COURSE_DATA címkék között találhatóak. '
                'A követelmények értelmezésének példája: 4/2/0/v (4 óra előadás/2 óra gyakorlat/0 óra labor/vizsga). '
                'Ha "v" betűvel végződik, a tárgy vizsgával zárul, "f" betű esetén félévközi értékeléssel. '
                'Egy félévben 30 kreditet kell teljesíteni. A tárgyak kreditértéke 1 és 15 között változhat. '
                'A feladatom kérdés-válasz párok generálása a megadott kontextus alapján JSON formátumban. '
                'Íme egy példa: '
            
                '"question": "Hány kreditet ér a "Számítógépes grafika" tárgy?", '
                '"answer": "A tantárgyi adatlapja alapján a "Számítógépes grafika" (VIIIAB07) tárgy 3 kreditet ér."'    
                                
                'A kérdések kontextusa a következő: {context} '
                'A kérdéseket azonnal megválaszolom a kontextus alapján. Minden kérdést és választ új sorban, JSON objektum formátumban adok meg. '
                'A kérdés-válasz párok száma: {count} '
                'A kérdések valószerüek olyanok amiket egy hallgató is feltenne az adott kurzussal kapcsolatban.'
                'Olyan legyen a kérdés aminek a kontextus ismerete nélkül is van értelme.'
                'A kérdések rövidek a válaszok hosszúak és informatívak.'
                'A kérdések a következők JSON listaként:'
            )
        )

    # Chain the QA template with the LLM for response generation
    chain = qa | llm

    # Invoke the chain to generate the final response
    result = chain.invoke({'context': context, "count":question_count}).content
    result = json.loads(result)
    result = {"context":context, "questions":result}
    return result
def generate_dataset(themes, question_count, context_count, path, llm, random_question_count=False):
    results = []
    for theme in themes:
        try:
            print(f"Processing theme: {theme}")
            result = generate_question_answer_context(theme, question_count=question_count, context_count=context_count, llm=llm)
            results.append(result)
        except Exception as e:
            print(f"Error processing theme {theme}: {e}")
            continue
    with open(path, 'w', encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)
            

In [5]:
themes = [
    "Analízis 1",
    "A programozás alapjai 1",
    "Bevezetés a számításelméletbe 1",
    "Bevezető fizika",
    "Bevezető matematika",
    "Digitális kompetenciák",
    "Digitális technika",
    "Fizika 1i",
    "Mérnök leszek",
    "Villamos alapismeretek",
    "Analízis 2",
    "Analízis szigorlat informatikusoknak",
    "A programozás alapjai 2",
    "Bevezetés a számításelméletbe 2",
    "Fizika 2i",
    "Rendszermodellezés",
    "Számítógép architektúrák",
    "A programozás alapjai 3",
    "Adatbázisok",
    "Kódolástechnika",
    "Kommunikációs hálózatok 1",
    "Rendszerelmélet",
    "Szoftvertechnológia",
    "Valószínűségszámítás",
    "Algoritmuselmélet",
    "Kommunikációs hálózatok 2",
    "Menedzsment és vállalkozásgazdaságtan",
    "Operációs rendszerek",
    "Számítógépes grafika",
    "Szoftver projekt laboratórium",
    "Szoftvertechnikák",
    "IT eszközök technológiája",
    "Mesterséges intelligencia",
    "Mikro- és makroökonómia",
    "Mobil- és webes szoftverek",
    "Üzleti jog",
    "Információs rendszerek üzemeltetése",
    "IT biztonság",
    ]

In [143]:
result = generate_question_answer_context(themes[0], 2, llm=llm_gpt4)

In [144]:
print(result)

{'context': 'BEGINING_OF_COURSE_DATA\nA kurzus címe: Analízis 1 informatikusoknak,\nKreditértéke: 6,\nKövetelmények: 4/2/0/v,\nForrás: https://portal.vik.bme.hu/kepzes/targyak/TE90AX21,\nA tárgy leírása: BME VIK - Analízis 1 informatikusoknak\nBudapest University of Technology and Economics, Faculty of Electrical Engineering and Informatics\nAnalízis 1 informatikusoknak\nBelépés\ncímtáras azonosítással\ntantárgyi adatlapok\nmagyar nyelvű adatlap\nvissza a tantárgylistához\nnyomtatható verzió\nAnalízis 1 informatikusoknak\nA tantárgy angol neve: Calculus 1 for Informaticians\nAdatlap utolsó módosítása: 2017. június 16.\nBudapesti Műszaki és Gazdaságtudományi Egyetem\nVillamosmérnöki és Informatikai Kar\nMérnökinformatikus szak, BSc képzés\nTantárgykód\nSzemeszter\nKövetelmények\nKredit\nTantárgyfélév\nTE90AX21\n1\n4/2/0/v\n6\n3. A tantárgyfelelős személy és tanszék\nDr. Tasnádi Tamás Péter,\n4. A tantárgy előadója\nNév:\nBeosztás:\nTanszék, Intézet:\nDr. Pataki Gergely\negyetemi adjunkt

In [146]:
generate_dataset(themes[10:], context_count=1, question_count=5, llm=llm_gpt4, path="./tad-qa-dataset/one-course-dataset-train.json")

Processing theme: Analízis 2
Processing theme: Analízis szigorlat informatikusoknak
Processing theme: A programozás alapjai 2
Processing theme: Bevezetés a számításelméletbe 2
Processing theme: Fizika 2i
Processing theme: Rendszermodellezés
Processing theme: Számítógép architektúrák
Processing theme: A programozás alapjai 3
Processing theme: Adatbázisok
Processing theme: Kódolástechnika
Processing theme: Kommunikációs hálózatok 1
Processing theme: Rendszerelmélet
Processing theme: Szoftvertechnológia
Processing theme: Valószínűségszámítás
Processing theme: Algoritmuselmélet
Processing theme: Kommunikációs hálózatok 2
Error processing theme Kommunikációs hálózatok 2: Unterminated string starting at: line 16 column 15 (char 735)
Processing theme: Menedzsment és vállalkozásgazdaságtan
Processing theme: Operációs rendszerek
Processing theme: Számítógépes grafika
Processing theme: Szoftver projekt laboratórium
Processing theme: Szoftvertechnikák
Processing theme: IT eszközök technológiája
P

In [11]:
path_to_test_set = "./tad-qa-dataset/one-course-dataset.json"

with open(path_to_test_set, "r", encoding="utf-8") as f:
    test_set = json.load(f)

In [12]:
for example in test_set[:]:
    context = example["context"]
    title_target = context.split("\n")[1][len("A kurzus címe: "):-1]
    
    for question in example["questions"]:
        query = question["question"]
        qa = PromptTemplate.from_template(
            template=(
                'A feladatod a megadott kérdésből egy rossz minőséű krédést generálni. A cél, hogy ne legyen tökéletes, olyan legyen mintha egy ember gépelte volna sietve'
                'Néhány ötlet: adjál hozzá helyesírási hibát, szlenget, elgépelést vagy csak fogalmazzad át a kérdést'
                'Példa: Mennyi kreditet ér az Analízis 1 informatikusoknak tárgy?'
                'Átirat: Hany kerdites anal 1?'
                'Példa: Milyen témaköröket tartalmaz az Analízis 1 informatikusoknak tárgy?'
                'Átirat: Miről szól analizis 1?'
                'Példa: {query}'
                'Átirat:'
            )
        )
        chain = qa | llm_gpt3
        result = chain.invoke({'query': query}).content
        question['difficult'] =result


In [13]:
with open("./tad-qa-dataset/one-course-dataset-difficult.json", 'w', encoding="utf-8") as f:
    json.dump(test_set, f, ensure_ascii=False, indent=4)